<a href="https://colab.research.google.com/github/yl4579/StyleTTS2/blob/main/Colab/StyleTTS2_Demo_LibriTTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install packages and download models

In [ ]:
%%shell
git clone https://github.com/yl4579/StyleTTS2.git
cd StyleTTS2
pip install SoundFile torchaudio munch torch pydub pyyaml librosa nltk matplotlib accelerate transformers phonemizer einops einops-exts tqdm typing-extensions git+https://github.com/resemble-ai/monotonic_align.git
sudo apt-get install espeak-ng
git-lfs clone https://huggingface.co/yl4579/StyleTTS2-LibriTTS
mv StyleTTS2-LibriTTS/Models .
mv StyleTTS2-LibriTTS/reference_audio.zip .
unzip reference_audio.zip
mv reference_audio Demo/reference_audio

In [15]:
%cd /workspace/styletts2-modded
!virtualenv venv && source venv/bin/activate
!pip install pandas

/workspace/styletts2-modded


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


created virtual environment CPython3.10.12.final.0-64 in 4158ms
  creator CPython3Posix(dest=/workspace/styletts2-modded/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: Markdown==3.7, MarkupSafe==3.0.2, PyYAML==6.0.2, absl_py==2.1.0, accelerate==1.3.0, attrs==25.1.0, audioread==3.0.1, awscli==1.37.11, babel==2.17.0, bibtexparser==2.0.0b8, botocore==1.36.11, certifi==2025.1.31, cffi==1.17.1, charset_normalizer==3.4.1, click==8.1.8, clldutils==3.24.1, colorama==0.4.6, colorlog==6.9.0, contourpy==1.3.1, csvw==3.5.1, cycler==0.12.1, decorator==5.1.1, dlinfo==2.0.0, docutils==0.16, einops==0.8.0, einops_exts==0.0.4, filelock==3.17.0, fonttools==4.55.8, fsspec==2025.2.0, grpcio==1.70.0, huggingface_hub==0.28.1, idna==3.10, isodate==0.7.2, jinja2==3.1.5, jmespath==1.0.1, joblib==1.4.2, jsonschema==4.23.0, jsonschema_specificatio

### Analyzing the Data

In [298]:
import os
import wave
import pandas as pd
from pathlib import Path

def get_wav_length_distribution(folder_path):
    """
    Analyze the length distribution of WAV files in a folder.
    
    Args:
        folder_path (str): Path to the folder containing WAV files
        
    Returns:
        None (prints distribution table)
    """
    # List to store lengths of all WAV files
    lengths = []
    
    # Walk through the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.wav'):
                try:
                    # Open WAV file
                    wav_path = os.path.join(root, file)
                    with wave.open(wav_path, 'rb') as wav_file:
                        # Calculate length in seconds
                        frames = wav_file.getnframes()
                        rate = wav_file.getframerate()
                        duration = round(frames / float(rate))
                        lengths.append(duration)
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
    
    if not lengths:
        print("No WAV files found in the specified folder.")
        return
    
    # Create distribution DataFrame
    min_length = min(lengths)
    max_length = max(lengths)
    
    # Initialize distribution dictionary
    distribution = {
        'Length (seconds)': range(min_length, max_length + 1),
        'Count': 0,
        'Percentage': 0.0,
        'Cumulative %': 0.0
    }
    
    # Count occurrences
    length_counts = pd.Series(lengths).value_counts().sort_index()
    
    # Create DataFrame
    df = pd.DataFrame(distribution)
    df = df.set_index('Length (seconds)')
    
    # Fill in counts and percentages
    total_files = len(lengths)
    cumulative_percent = 0
    
    for length in length_counts.index:
        count = length_counts[length]
        percentage = (count / total_files) * 100
        cumulative_percent += percentage
        
        df.loc[length, 'Count'] = count
        df.loc[length, 'Percentage'] = percentage
        df.loc[length, 'Cumulative %'] = cumulative_percent
    
    # Fill NaN values with 0
    df = df.fillna(0)
    
    # Format percentages to 2 decimal places
    df['Percentage'] = df['Percentage'].round(2)
    df['Cumulative %'] = df['Cumulative %'].round(2)
    
    # Add total row
    df.loc['Total'] = [total_files, 100.0, 100.0]
    
    # Print statistics
    print(f"\nTotal number of WAV files: {total_files}")
    print(f"Minimum length: {min_length} seconds")
    print(f"Maximum length: {max_length} seconds")
    print(f"Average length: {round(sum(lengths)/len(lengths), 2)} seconds")
    print("\nLength Distribution:")
    print(df)

# Example usage
if __name__ == "__main__":
    # Replace with your folder path
    folder_path = "/workspace/StyleTTS2/Data_Tony/audio_data/"
    get_wav_length_distribution(folder_path)


Total number of WAV files: 997
Minimum length: 0 seconds
Maximum length: 22 seconds
Average length: 4.42 seconds

Length Distribution:
                  Count  Percentage  Cumulative %
Length (seconds)                                 
0                   9.0        0.90          0.90
1                 244.0       24.47         25.38
2                 183.0       18.36         43.73
3                 115.0       11.53         55.27
4                  88.0        8.83         64.09
5                  64.0        6.42         70.51
6                  58.0        5.82         76.33
7                  44.0        4.41         80.74
8                  42.0        4.21         84.95
9                  35.0        3.51         88.47
10                 27.0        2.71         91.17
11                 20.0        2.01         93.18
12                 19.0        1.91         95.09
13                 15.0        1.50         96.59
14                 10.0        1.00         97.59
15            

In [6]:
import os
import wave
import pandas as pd
from pathlib import Path

def get_wav_length_distribution(folder_path):
    """
    Analyze the length distribution of WAV files in a folder.
    
    Args:
        folder_path (str): Path to the folder containing WAV files
        
    Returns:
        None (prints distribution table)
    """
    # List to store lengths of all WAV files
    lengths = []
    
    # Walk through the folder
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.wav'):
                try:
                    # Open WAV file
                    wav_path = os.path.join(root, file)
                    with wave.open(wav_path, 'rb') as wav_file:
                        # Calculate length in seconds
                        frames = wav_file.getnframes()
                        rate = wav_file.getframerate()
                        duration = round(frames / float(rate))
                        lengths.append(duration)
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
    
    if not lengths:
        print("No WAV files found in the specified folder.")
        return
    
    # Create distribution DataFrame
    min_length = min(lengths)
    max_length = max(lengths)
    
    # Initialize distribution dictionary
    distribution = {
        'Length (seconds)': range(min_length, max_length + 1),
        'Count': 0,
        'Percentage': 0.0,
        'Cumulative %': 0.0
    }
    
    # Count occurrences
    length_counts = pd.Series(lengths).value_counts().sort_index()
    
    # Create DataFrame
    df = pd.DataFrame(distribution)
    df = df.set_index('Length (seconds)')
    
    # Fill in counts and percentages
    total_files = len(lengths)
    cumulative_percent = 0
    
    for length in length_counts.index:
        count = length_counts[length]
        percentage = (count / total_files) * 100
        cumulative_percent += percentage
        
        df.loc[length, 'Count'] = count
        df.loc[length, 'Percentage'] = percentage
        df.loc[length, 'Cumulative %'] = cumulative_percent
    
    # Fill NaN values with 0
    df = df.fillna(0)
    
    # Format percentages to 2 decimal places
    df['Percentage'] = df['Percentage'].round(2)
    df['Cumulative %'] = df['Cumulative %'].round(2)
    
    # Add total row
    df.loc['Total'] = [total_files, 100.0, 100.0]
    
    # Print statistics
    print(f"\nTotal number of WAV files: {total_files}")
    print(f"Minimum length: {min_length} seconds")
    print(f"Maximum length: {max_length} seconds")
    print(f"Average length: {round(sum(lengths)/len(lengths), 2)} seconds")
    print("\nLength Distribution:")
    print(df)

# Example usage
if __name__ == "__main__":
    # Replace with your folder path
    # folder_path = "/workspace/StyleTTS2/Data_Tony/audio_data/"
    folder_path = "/workspace/styletts2-modded/dataset__10000_11labs_tony_tc/audio_data/"
    get_wav_length_distribution(folder_path)

Error processing elevenlabs_0720.wav: file does not start with RIFF id
Error processing elevenlabs_0722.wav: file does not start with RIFF id
Error processing elevenlabs_0658.wav: file does not start with RIFF id
Error processing elevenlabs_0633.wav: file does not start with RIFF id
Error processing elevenlabs_0595.wav: file does not start with RIFF id
Error processing elevenlabs_0581.wav: file does not start with RIFF id
Error processing elevenlabs_0498.wav: file does not start with RIFF id
Error processing elevenlabs_0486.wav: file does not start with RIFF id
Error processing elevenlabs_0482.wav: file does not start with RIFF id
Error processing elevenlabs_0480.wav: file does not start with RIFF id
Error processing elevenlabs_0410.wav: file does not start with RIFF id
Error processing elevenlabs_0387.wav: file does not start with RIFF id
Error processing elevenlabs_0349.wav: file does not start with RIFF id
Error processing elevenlabs_0332.wav: file does not start with RIFF id
Error 

### Merging Datasets

In [13]:
import os
import shutil
import pandas as pd
import random

def merge_folders(folder1_path, folder2_path, output_folder_path, n_files_folder1=500,
                 prefix1="folder1_", prefix2="folder2_"):
    """
    Merge two folders containing metadata.csv and audio_data/, taking only the first
    n_files_folder1 from folder1, adding prefixes to files, and shuffling the metadata entries.
    
    Args:
        folder1_path (str): Path to first folder
        folder2_path (str): Path to second folder
        output_folder_path (str): Path where merged folder will be created
        n_files_folder1 (int): Number of files to take from folder1
        prefix1 (str): Prefix to add to files from folder1
        prefix2 (str): Prefix to add to files from folder2
    """
    
    # Create output folder and its audio_data subdirectory if they don't exist
    os.makedirs(output_folder_path, exist_ok=True)
    os.makedirs(os.path.join(output_folder_path, 'audio_data'), exist_ok=True)
    
    # Read metadata files
    metadata1 = pd.read_csv(os.path.join(folder1_path, 'metadata.csv'), 
                           header=None, sep='|')
    metadata2 = pd.read_csv(os.path.join(folder2_path, 'metadata.csv'), 
                           header=None, sep='|')
    
    # Take only first n_files_folder1 from folder1
    metadata1 = metadata1.head(n_files_folder1)
    
    # Add prefixes to filenames in metadata
    def add_prefix(df, prefix):
        df = df.copy()
        df[0] = df[0].apply(lambda x: os.path.join('audio_data', 
                                                  prefix + os.path.basename(x)))
        return df

    metadata1 = add_prefix(metadata1, prefix1)
    metadata2 = add_prefix(metadata2, prefix2)
    
    # Combine metadata
    combined_metadata = pd.concat([metadata1, metadata2])
    
    # Shuffle the combined metadata
    combined_metadata = combined_metadata.sample(frac=1).reset_index(drop=True)
    
    # Copy audio files and save metadata
    for _, row in combined_metadata.iterrows():
        new_audio_file = os.path.basename(row[0])  # Get the new filename with prefix
        original_audio_file = new_audio_file
        
        # Remove the prefix to find the original file
        if original_audio_file.startswith(prefix1):
            source_folder = folder1_path
            original_audio_file = original_audio_file[len(prefix1):]
        else:
            source_folder = folder2_path
            original_audio_file = original_audio_file[len(prefix2):]
            
        # Copy audio file
        source_path = os.path.join(source_folder, 'audio_data', original_audio_file)
        dest_path = os.path.join(output_folder_path, 'audio_data', new_audio_file)
        shutil.copy2(source_path, dest_path)
    
    # Save shuffled metadata
    combined_metadata.to_csv(os.path.join(output_folder_path, 'metadata.csv'), 
                           header=False, index=False, sep='|')
    
    # Print some statistics
    print(f"Number of files from folder1: {len(metadata1)}")
    print(f"Number of files from folder2: {len(metadata2)}")
    print(f"Total files in output: {len(combined_metadata)}")

# Example usage:
merge_folders(
    '/workspace/styletts2-modded/Data_Tony/',
    '/workspace/styletts2-modded/dataset__10000_11labs_tony_tc',
    '/workspace/styletts2-modded/data_tony_all',
    prefix1='Data_Tony_',
    prefix2='data_tc',
    n_files_folder1=600
)


Number of files from folder1: 600
Number of files from folder2: 742
Total files in output: 1342


In [14]:
5

5

### Load models

In [17]:
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [19]:
# %cd StyleTTS2
# %cd ../

import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("/workspace/StyleTTS2/Models/Data_Tony_15epochs/config.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

# params_whole = torch.load("/workspace/StyleTTS2/Models/Data_Tony/epoch_2nd_00003.pth", map_location='cpu')
# params_whole = torch.load("/workspace/StyleTTS2/Models/Data_Tony_10epochs/epoch_2nd_00002.pth", map_location='cpu')
# params_whole = torch.load("/workspace/StyleTTS2/Models/Data_Tony_10epochs/epoch_2nd_00004.pth", map_location='cpu')
# params_whole = torch.load("/workspace/StyleTTS2/Models/Data_Tony_10epochs/epoch_2nd_00008.pth", map_location='cpu')
params_whole = torch.load("/workspace/StyleTTS2/Models/Data_Tony_15epochs/epoch_2nd_00005.pth", map_location='cpu')
params = params_whole['net']

for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)


        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

def LFinference(text, s_prev, ref_s, alpha = 0.3, beta = 0.7, t = 0.7, diffusion_steps=5, embedding_scale=1):
  text = text.strip()
  ps = global_phonemizer.phonemize([text])
  ps = word_tokenize(ps[0])
  ps = ' '.join(ps)
  ps = ps.replace('``', '"')
  ps = ps.replace("''", '"')

  tokens = textclenaer(ps)
  tokens.insert(0, 0)
  tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

  with torch.no_grad():
      input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
      text_mask = length_to_mask(input_lengths).to(device)

      t_en = model.text_encoder(tokens, input_lengths, text_mask)
      bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
      d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

      s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                        embedding=bert_dur,
                                        embedding_scale=embedding_scale,
                                          features=ref_s, # reference from the same speaker as the embedding
                                            num_steps=diffusion_steps).squeeze(1)

      if s_prev is not None:
          # convex combination of previous and current style
          s_pred = t * s_prev + (1 - t) * s_pred

      s = s_pred[:, 128:]
      ref = s_pred[:, :128]

      ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
      s = beta * s + (1 - beta)  * ref_s[:, 128:]

      s_pred = torch.cat([ref, s], dim=-1)

      d = model.predictor.text_encoder(d_en,
                                        s, input_lengths, text_mask)

      x, _ = model.predictor.lstm(d)
      duration = model.predictor.duration_proj(x)

      duration = torch.sigmoid(duration).sum(axis=-1)
      pred_dur = torch.round(duration.squeeze()).clamp(min=1)


      pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
      c_frame = 0
      for i in range(pred_aln_trg.size(0)):
          pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
          c_frame += int(pred_dur[i].data)

      # encode prosody
      en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
      if model_params.decoder.type == "hifigan":
          asr_new = torch.zeros_like(en)
          asr_new[:, :, 0] = en[:, :, 0]
          asr_new[:, :, 1:] = en[:, :, 0:-1]
          en = asr_new

      F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

      asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
      if model_params.decoder.type == "hifigan":
          asr_new = torch.zeros_like(asr)
          asr_new[:, :, 0] = asr[:, :, 0]
          asr_new[:, :, 1:] = asr[:, :, 0:-1]
          asr = asr_new

      out = model.decoder(asr,
                              F0_pred, N_pred, ref.squeeze().unsqueeze(0))


  return out.squeeze().cpu().numpy()[..., :-100], s_pred # weird pulse at the end of the model, need to be fixed later

def STinference(text, ref_s, ref_text, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    ref_text = ref_text.strip()
    ps = global_phonemizer.phonemize([ref_text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)

    ref_tokens = textclenaer(ps)
    ref_tokens.insert(0, 0)
    ref_tokens = torch.LongTensor(ref_tokens).to(device).unsqueeze(0)


    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        ref_input_lengths = torch.LongTensor([ref_tokens.shape[-1]]).to(device)
        ref_text_mask = length_to_mask(ref_input_lengths).to(device)
        ref_bert_dur = model.bert(ref_tokens, attention_mask=(~ref_text_mask).int())
        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)


        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later


ModuleNotFoundError: No module named 'munch'

### Synthesize speech

#### Basic synthesis (5 diffusion steps, seen speakers)

In [291]:
text = '''Intriguing, my friend, so there's something specific that draws you to the Catholic Church, but it's not necessarily the traditional values or teachings - can you tell me more about what that "something else" is, is it a personal experience, a sense of mystery, or perhaps a desire to serve in a unique way?''' # @param {type:"string"}
# text = "That's a bold vision."
# text = "Hi, I'm Tony! What's up?"


In [277]:
reference_dicts = {}
reference_dicts['696_92939'] = "/workspace/StyleTTS2/Demo/rebel_spaceships.mp3"

In [211]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=2)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

RTF = 0.089993, 247.3471164703369ms
696_92939 Synthesized:


Reference:


In [219]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=2)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

RTF = 0.194279, 490.19694328308105ms
module.detector.bias Synthesized:


Reference:


In [227]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=2)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

RTF = 0.170334, 438.29941749572754ms
module.detector.bias Synthesized:


Reference:


In [243]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.1, beta=0.9, diffusion_steps=5, embedding_scale=2)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

ɪntɹˈiːɡɪŋ , maɪ fɹˈɛnd , sˌoʊ ðɛɹz sˈʌmθɪŋ spəsˈɪfɪk ðæt dɹˈɔːz juː tə ðə kˈæθlɪk tʃˈɜːtʃ , bˌʌt ɪts nˌɑːt nˌɛsɪsˈɛɹᵻli ðə tɹɐdˈɪʃənəl vˈæljuːz ɔːɹ tˈiːtʃɪŋz kæn juː tˈɛl mˌiː mˈoːɹ ɐbˌaʊt wʌt ðˈæt `` sˈʌmθɪŋ ˈɛls '' ɪz , ɪz ɪɾ ɐ pˈɜːsənəl ɛkspˈiəɹɪəns , ɐ sˈɛns ʌv mˈɪstɚɹi , ɔːɹ pɚhˈæps ɐ dɪzˈaɪɚ tə sˈɜːv ɪn ɐ juːnˈiːk wˈeɪ ?
ɪntɹˈiːɡɪŋ , maɪ fɹˈɛnd , sˌoʊ ðɛɹz sˈʌmθɪŋ spəsˈɪfɪk ðæt dɹˈɔːz juː tə ðə kˈæθlɪk tʃˈɜːtʃ , bˌʌt ɪts nˌɑːt nˌɛsɪsˈɛɹᵻli ðə tɹɐdˈɪʃənəl vˈæljuːz ɔːɹ tˈiːtʃɪŋz kæn juː tˈɛl mˌiː mˈoːɹ ɐbˌaʊt wʌt ðˈæt `` sˈʌmθɪŋ ˈɛls '' ɪz , ɪz ɪɾ ɐ pˈɜːsənəl ɛkspˈiəɹɪəns , ɐ sˈɛns ʌv mˈɪstɚɹi , ɔːɹ pɚhˈæps ɐ dɪzˈaɪɚ tə sˈɜːv ɪn ɐ juːnˈiːk wˈeɪ ?
RTF = 0.054777, 895.5438137054443ms
module.detector.bias Synthesized:


Reference:


In [285]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.6, beta=0.5, diffusion_steps=5, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

RTF = 0.085439, 211.33708953857422ms
module.detector.bias Synthesized:


Reference:


In [292]:
# noise = torch.randn(1,1,256).to(device)
# for k, path in reference_dicts.items():
    # ref_s = compute_style(path)
start = time.time()
wav = inference(text, ref_s, alpha=0.05, beta=0.95, diffusion_steps=7, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}, {(time.time() - start) * 1000}ms")
import IPython.display as ipd
print(k + ' Synthesized:')
display(ipd.Audio(wav, rate=24000, normalize=False))
print('Reference:')
display(ipd.Audio(path, rate=24000, normalize=False))

ɪntɹˈiːɡɪŋ , maɪ fɹˈɛnd , sˌoʊ ðɛɹz sˈʌmθɪŋ spəsˈɪfɪk ðæt dɹˈɔːz juː tə ðə kˈæθlɪk tʃˈɜːtʃ , bˌʌt ɪts nˌɑːt nˌɛsɪsˈɛɹᵻli ðə tɹɐdˈɪʃənəl vˈæljuːz ɔːɹ tˈiːtʃɪŋz kæn juː tˈɛl mˌiː mˈoːɹ ɐbˌaʊt wʌt ðˈæt `` sˈʌmθɪŋ ˈɛls '' ɪz , ɪz ɪɾ ɐ pˈɜːsənəl ɛkspˈiəɹɪəns , ɐ sˈɛns ʌv mˈɪstɚɹi , ɔːɹ pɚhˈæps ɐ dɪzˈaɪɚ tə sˈɜːv ɪn ɐ juːnˈiːk wˈeɪ ?
ɪntɹˈiːɡɪŋ , maɪ fɹˈɛnd , sˌoʊ ðɛɹz sˈʌmθɪŋ spəsˈɪfɪk ðæt dɹˈɔːz juː tə ðə kˈæθlɪk tʃˈɜːtʃ , bˌʌt ɪts nˌɑːt nˌɛsɪsˈɛɹᵻli ðə tɹɐdˈɪʃənəl vˈæljuːz ɔːɹ tˈiːtʃɪŋz kæn juː tˈɛl mˌiː mˈoːɹ ɐbˌaʊt wʌt ðˈæt `` sˈʌmθɪŋ ˈɛls '' ɪz , ɪz ɪɾ ɐ pˈɜːsənəl ɛkspˈiəɹɪəns , ɐ sˈɛns ʌv mˈɪstɚɹi , ɔːɹ pɚhˈæps ɐ dɪzˈaɪɚ tə sˈɜːv ɪn ɐ juːnˈiːk wˈeɪ ?
RTF = 0.025197, 454.16712760925293ms
module.detector.bias Synthesized:


Reference:


#### With higher diffusion steps (more diverse)

Since the sampler is ancestral, the higher the stpes, the more diverse the samples are, with the cost of slower synthesis speed.

In [ ]:
noise = torch.randn(1,1,256).to(device)
for k, path in reference_dicts.items():
    ref_s = compute_style(path)
    start = time.time()
    wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=10, embedding_scale=1)
    rtf = (time.time() - start) / (len(wav) / 24000)
    print(f"RTF = {rtf:5f}")
    import IPython.display as ipd
    print(k + ' Synthesized:')
    display(ipd.Audio(wav, rate=24000, normalize=False))
    print(k + ' Reference:')
    display(ipd.Audio(path, rate=24000, normalize=False))

#### Basic synthesis (5 diffusion steps, unseen speakers)
The following samples are to reproduce samples in [Section 4](https://styletts2.github.io/#libri) of the demo page. All spsakers are unseen during training. You can compare the generated samples to popular zero-shot TTS models like Vall-E and NaturalSpeech 2.

In [ ]:
reference_dicts = {}
# format: (path, text)
reference_dicts['1221-135767'] = ("Demo/reference_audio/1221-135767-0014.wav", "Yea, his honourable worship is within, but he hath a godly minister or two with him, and likewise a leech.")
reference_dicts['5639-40744'] = ("Demo/reference_audio/5639-40744-0020.wav", "Thus did this humane and right minded father comfort his unhappy daughter, and her mother embracing her again, did all she could to soothe her feelings.")
reference_dicts['908-157963'] = ("Demo/reference_audio/908-157963-0027.wav", "And lay me down in my cold bed and leave my shining lot.")
reference_dicts['4077-13754'] = ("Demo/reference_audio/4077-13754-0000.wav", "The army found the people in poverty and left them in comparative wealth.")

In [ ]:
noise = torch.randn(1,1,256).to(device)
for k, v in reference_dicts.items():
    path, text = v
    ref_s = compute_style(path)
    start = time.time()
    wav = inference(text, ref_s, alpha=0.3, beta=0.7, diffusion_steps=5, embedding_scale=1)
    rtf = (time.time() - start) / (len(wav) / 24000)
    print(f"RTF = {rtf:5f}")
    import IPython.display as ipd
    print(k + ' Synthesized: ' + text)
    display(ipd.Audio(wav, rate=24000, normalize=False))
    print(k + ' Reference:')
    display(ipd.Audio(path, rate=24000, normalize=False))

### Speech expressiveness

The following section recreates the samples shown in [Section 6](https://styletts2.github.io/#emo) of the demo page. The speaker reference used is `1221-135767-0014.wav`, which is unseen during training.

#### With `embedding_scale=1`
This is the classifier-free guidance scale. The higher the scale, the more conditional the style is to the input text and hence more emotional.

In [ ]:
ref_s = compute_style("Demo/reference_audio/1221-135767-0014.wav")

In [ ]:
texts = {}
texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

for k,v in texts.items():
    wav = inference(v, ref_s, diffusion_steps=10, alpha=0.3, beta=0.7, embedding_scale=1)
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### With `embedding_scale=2`

In [ ]:
texts = {}
texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

for k,v in texts.items():
    noise = torch.randn(1,1,256).to(device)
    wav = inference(v, ref_s, diffusion_steps=10, alpha=0.3, beta=0.7, embedding_scale=2)
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### With `embedding_scale=2, alpha = 0.5, beta = 0.9`
`alpha` and `beta` is the factor to determine much we use the style sampled based on the text instead of the reference. The higher the value of `alpha` and `beta`, the more suitable the style it is to the text but less similar to the reference. Using higher beta makes the synthesized speech more emotional, at the cost of lower similarity to the reference. `alpha` determines the timbre of the speaker while `beta` determines the prosody.

In [ ]:
texts = {}
texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

for k,v in texts.items():
    noise = torch.randn(1,1,256).to(device)
    wav = inference(v, ref_s, diffusion_steps=10, alpha=0.5, beta=0.9, embedding_scale=2)
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

### Zero-shot speaker adaptation
This section recreates the "Acoustic Environment Maintenance" and "Speaker’s Emotion Maintenance" demo in [Section 4](https://styletts2.github.io/#libri) of the demo page. You can compare the generated samples to popular zero-shot TTS models like Vall-E. Note that the model was trained only on LibriTTS, which is about 250 times fewer data compared to those used to trian Vall-E with similar or better effect for these maintainance.

#### Acoustic Environment Maintenance

Since we want to maintain the acoustic environment in the speaker (timbre), we set  `alpha = 0` to make the speaker as close to the reference as possible while only changing the prosody according to the text.  

In [ ]:
reference_dicts = {}
# format: (path, text)
reference_dicts['3'] = ("Demo/reference_audio/3.wav", "As friends thing I definitely I've got more male friends.")
reference_dicts['4'] = ("Demo/reference_audio/4.wav", "Everything is run by computer but you got to know how to think before you can do a computer.")
reference_dicts['5'] = ("Demo/reference_audio/5.wav", "Then out in LA you guys got a whole another ball game within California to worry about.")

In [ ]:
noise = torch.randn(1,1,256).to(device)
for k, v in reference_dicts.items():
    path, text = v
    ref_s = compute_style(path)
    start = time.time()
    wav = inference(text, ref_s, alpha=0.0, beta=0.5, diffusion_steps=5, embedding_scale=1)
    rtf = (time.time() - start) / (len(wav) / 24000)
    print(f"RTF = {rtf:5f}")
    import IPython.display as ipd
    print('Synthesized: ' + text)
    display(ipd.Audio(wav, rate=24000, normalize=False))
    print('Reference:')
    display(ipd.Audio(path, rate=24000, normalize=False))

#### Speaker’s Emotion Maintenance

Since we want to maintain the emotion in the speaker (prosody), we set  `beta = 0.1` to make the speaker as closer to the reference as possible while having some diversity thruogh the slight timbre change.

In [ ]:
reference_dicts = {}
# format: (path, text)
reference_dicts['Anger'] = ("Demo/reference_audio/anger.wav", "We have to reduce the number of plastic bags.")
reference_dicts['Sleepy'] = ("Demo/reference_audio/sleepy.wav", "We have to reduce the number of plastic bags.")
reference_dicts['Amused'] = ("Demo/reference_audio/amused.wav", "We have to reduce the number of plastic bags.")
reference_dicts['Disgusted'] = ("Demo/reference_audio/disgusted.wav", "We have to reduce the number of plastic bags.")

In [ ]:
noise = torch.randn(1,1,256).to(device)
for k, v in reference_dicts.items():
    path, text = v
    ref_s = compute_style(path)
    start = time.time()
    wav = inference(text, ref_s, alpha=0.3, beta=0.1, diffusion_steps=10, embedding_scale=1)
    rtf = (time.time() - start) / (len(wav) / 24000)
    print(f"RTF = {rtf:5f}")
    import IPython.display as ipd
    print(k + ' Synthesized: ' + text)
    display(ipd.Audio(wav, rate=24000, normalize=False))
    print(k + ' Reference:')
    display(ipd.Audio(path, rate=24000, normalize=False))

### Longform Narration

This section includes basic implementation of Algorithm 1 in the paper for consistent longform audio generation. The example passage is taken from [Section 5](https://styletts2.github.io/#long) of the demo page.

In [ ]:
passage = passage = '''If the supply of fruit is greater than the family needs, it may be made a source of income by sending the fresh fruit to the market if there is one near enough, or by preserving, canning, and making jelly for sale. To make such an enterprise a success the fruit and work must be first class. There is magic in the word "Homemade," when the product appeals to the eye and the palate; but many careless and incompetent people have found to their sorrow that this word has not magic enough to float inferior goods on the market. As a rule large canning and preserving establishments are clean and have the best appliances, and they employ chemists and skilled labor. The home product must be very good to compete with the attractive goods that are sent out from such establishments. Yet for first class home made products there is a market in all large cities. All first-class grocers have customers who purchase such goods.''' # @param {type:"string"}

In [ ]:
# seen speaker
path = "Demo/reference_audio/696_92939_000016_000006.wav"
s_ref = compute_style(path)
sentences = passage.split('.') # simple split by comma
wavs = []
s_prev = None
for text in sentences:
    if text.strip() == "": continue
    text += '.' # add it back

    wav, s_prev = LFinference(text,
                              s_prev,
                              s_ref,
                              alpha = 0.3,
                              beta = 0.9,  # make it more suitable for the text
                              t = 0.7,
                              diffusion_steps=10, embedding_scale=1.5)
    wavs.append(wav)
print('Synthesized: ')
display(ipd.Audio(np.concatenate(wavs), rate=24000, normalize=False))
print('Reference: ')
display(ipd.Audio(path, rate=24000, normalize=False))

### Style Transfer

The following section demostrates the style transfer capacity for unseen speakers in [Section 6](https://styletts2.github.io/#emo) of the demo page. For this, we set `alpha=0.5, beta = 0.9` for the most pronounced effects (mostly using the sampled style).

In [ ]:
# reference texts to sample styles

ref_texts = {}
ref_texts['Happy'] = "We are happy to invite you to join us on a journey to the past, where we will visit the most amazing monuments ever built by human hands."
ref_texts['Sad'] = "I am sorry to say that we have suffered a severe setback in our efforts to restore prosperity and confidence."
ref_texts['Angry'] = "The field of astronomy is a joke! Its theories are based on flawed observations and biased interpretations!"
ref_texts['Surprised'] = "I can't believe it! You mean to tell me that you have discovered a new species of bacteria in this pond?"

In [ ]:
path = "Demo/reference_audio/1221-135767-0014.wav"
s_ref = compute_style(path)

text = "Yea, his honourable worship is within, but he hath a godly minister or two with him, and likewise a leech."
for k,v in ref_texts.items():
    wav = STinference(text, s_ref, v, diffusion_steps=10, alpha=0.5, beta=0.9, embedding_scale=1.5)
    print(k + ": ")
    display(ipd.Audio(wav, rate=24000, normalize=False))

### Speech diversity

This section reproduces samples in [Section 7](https://styletts2.github.io/#var) of the demo page.

`alpha` and `beta` determine the diversity of the synthesized speech. There are two extreme cases:
- If `alpha = 1` and `beta = 1`, the synthesized speech sounds the most dissimilar to the reference speaker, but it is also the most diverse (each time you synthesize a speech it will be totally different).
- If `alpha = 0` and `beta = 0`, the synthesized speech sounds the most siimlar to the reference speaker, but it is deterministic (i.e., the sampled style is not used for speech synthesis).


#### Default setting (`alpha = 0.3, beta=0.7`)
This setting uses 70% of the reference timbre and 30% of the reference prosody and use the diffusion model to sample them based on the text.

In [ ]:
# unseen speaker
path = "Demo/reference_audio/1221-135767-0014.wav"
ref_s = compute_style(path)

text = "How much variation is there?"
for _ in range(5):
    wav = inference(text, ref_s, diffusion_steps=10, alpha=0.3, beta=0.7, embedding_scale=1)
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### Less diverse setting (`alpha = 0.1, beta=0.3`)
This setting uses 90% of the reference timbre and 70% of the reference prosody. This makes it more similar to the reference speaker at cost of less diverse samples.

In [ ]:
# unseen speaker
path = "Demo/reference_audio/1221-135767-0014.wav"
ref_s = compute_style(path)

text = "How much variation is there?"
for _ in range(5):
    wav = inference(text, ref_s, diffusion_steps=10, alpha=0.1, beta=0.3, embedding_scale=1)
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### More diverse setting (`alpha = 0.5, beta=0.95`)
This setting uses 50% of the reference timbre and 5% of the reference prosody (so it uses 100% of the sampled prosody, which makes it more diverse), but this makes it more dissimilar to the reference speaker.  

In [ ]:
# unseen speaker
path = "Demo/reference_audio/1221-135767-0014.wav"
ref_s = compute_style(path)

text = "How much variation is there?"
for _ in range(5):
    wav = inference(text, ref_s, diffusion_steps=10, alpha=0.5, beta=0.95, embedding_scale=1)
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### Extreme setting (`alpha = 1, beta=1`)
This setting uses 0% of the reference timbre and prosody and use the diffusion model to sample the entire style. This makes the speaker very dissimilar to the reference speaker.

In [ ]:
# unseen speaker
path = "Demo/reference_audio/1221-135767-0014.wav"
ref_s = compute_style(path)

text = "How much variation is there?"
for _ in range(5):
    wav = inference(text, ref_s, diffusion_steps=10, alpha=1, beta=1, embedding_scale=1)
    display(ipd.Audio(wav, rate=24000, normalize=False))

#### No variation (`alpha = 0, beta=0`)
This setting uses 100% of the reference timbre and prosody and do not use the diffusion model at all. This makes the speaker very similar to the reference speaker, but there is no variation.

In [ ]:
# unseen speaker
path = "Demo/reference_audio/1221-135767-0014.wav"
ref_s = compute_style(path)

text = "How much variation is there?"
for _ in range(5):
    wav = inference(text, ref_s, diffusion_steps=10, alpha=0, beta=0, embedding_scale=1)
    display(ipd.Audio(wav, rate=24000, normalize=False))

### Extra fun!

You can record your own voice and clone it using pre-trained StyleTTS 2 model here.

#### Run the following cell to record your voice for 5 seconds. Please keep speaking to have the best effect.

In [ ]:
# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  return 'audio.wav'  # or webm ?

#### Please run this cell and speak:

In [ ]:
print('Speak now for 5 seconds.')
audio = record(sec=5)
import IPython.display as ipd
display(ipd.Audio(audio, rate=24000, normalize=False))

#### Synthesize in your own voice

In [ ]:
text = ''' StyleTTS 2 is a text to speech model that leverages style diffusion and adversarial training with large speech language models to achieve human level text to speech synthesis. ''' # @param {type:"string"}


In [ ]:
reference_dicts = {}
reference_dicts['You'] = audio

In [ ]:
start = time.time()
noise = torch.randn(1,1,256).to(device)
for k, path in reference_dicts.items():
    ref_s = compute_style(path)

    wav = inference(text, ref_s, alpha=0.1, beta=0.5, diffusion_steps=5, embedding_scale=1)
    rtf = (time.time() - start) / (len(wav) / 24000)
    print('Speaker: ' + k)
    import IPython.display as ipd
    print('Synthesized:')
    display(ipd.Audio(wav, rate=24000, normalize=False))
    print('Reference:')
    display(ipd.Audio(path, rate=24000, normalize=False))